In [ ]:
# nuclio: ignore
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter
import nuclio 

In [ ]:
%nuclio cmd pip install v3io_frames


In [ ]:
import v3io_frames as v3f
from datetime import datetime, timedelta

def handler(context, event):
    KV_TABLE_PATH = 'iguazio/demos/testkv'
    TSDB_TABLE_PATH = 'iguazio/demos/testtsdb'
    client = v3f.Client("framesd:8081", user='iguazio', password='datal@ke!' , container="users")
    #client.create("tsdb", TSDB_TABLE_PATH, attrs={"rate": "1/m"})
    interval = 1
    today = datetime.utcnow()
    delta = timedelta(hours=interval)
    delete_date = str((today - delta).timestamp()).replace(".",":")
    print(delete_date)





    df = client.read(backend='kv', table=KV_TABLE_PATH,filter="time<"+delete_date)
    print(df.columns)
    df.reset_index(inplace=True)
    df.set_index(['time'], inplace=True)
    df2 = df[['fileName','camera','label','imgUrl']]
    client.write("tsdb",TSDB_TABLE_PATH,dfs=df2)
    print(df2)


In [ ]:
# nuclio: ignore
event = nuclio.Event(body='b;good morninng')
handler(context, event)

Before deploying the function

Open up the Jupyter shell and run the following
pip install --upgrade nuclio-jupyter

In [ ]:
%nuclio deploy -n kv_to_tsdb -p test -c